In [1]:
%pip install --upgrade pip
!pip3 install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 4.0 MB/s eta 0:00:0000:0100:01


In [2]:
import polars as pl

In [5]:
df = pl.scan_parquet("../../data/commits.parquet")
df

In [11]:
df.columns

/var/folders/3b/kmz412j17b5c7z57yc6k1jmm0000gn/T/ipykernel_11789/3289350417.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df.columns


['commit',
 'message',
 'branch',
 'committer_name',
 'committer_email',
 'commit_dt',
 'author_name',
 'author_email',
 'author_dt',
 'parents']

In [9]:
result = df.select(
    pl.col("commit").n_unique().alias("n_unique"),
)
result.collect()

n_unique
u32
1724


In [24]:
result = df.select(
    pl.col("commit"),
    pl.col("parents")
).filter(pl.col("parents").list.len() > 1)
result.collect().count

<bound method DataFrame.count of shape: (184, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ commit                          ┆ parents                         │
│ ---                             ┆ ---                             │
│ str                             ┆ list[str]                       │
╞═════════════════════════════════╪═════════════════════════════════╡
│ 9c2821d6f04d24b7b6358779acb461… ┆ ["11cc3375e0f3df846f5e02785567… │
│ 11cc3375e0f3df846f5e02785567ed… ┆ ["b643e616095f1f303bb809a0227f… │
│ 6cdfd9294d374a69e233e2aa14a41c… ┆ ["a50bf95e13e1d036720839504390… │
│ b643e616095f1f303bb809a0227f28… ┆ ["d7cffcabe06fa2b2a854598a7af2… │
│ a50bf95e13e1d03672083950439041… ┆ ["476871addc4a085077a460933881… │
│ …                               ┆ …                               │
│ 5df9582b937395b2d1c40d3ce05486… ┆ ["fec41a6fc3e72606a707201479fe… │
│ 59403507ca8e2c324dda6a96f056f0… ┆ ["a3816559b93e73b589326643bbb5… │
│ feb52b2f9c49209fd45815605890ec… ┆ ["890

In [31]:
result = df.select(
    pl.col("commit"),
    pl.col("parents")
).filter(pl.col("parents").list.len() == 1)
result.collect().count

<bound method DataFrame.count of shape: (1_539, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ commit                          ┆ parents                         │
│ ---                             ┆ ---                             │
│ str                             ┆ list[str]                       │
╞═════════════════════════════════╪═════════════════════════════════╡
│ 9033b77b421711345f0da471ee5799… ┆ ["6c259676c6afa339a308ffa16f8f… │
│ 4980b87058ac407e55ff753c9a9cca… ┆ ["476871addc4a085077a460933881… │
│ 6c259676c6afa339a308ffa16f8f8f… ┆ ["293e134b0846ad77601d018b42e0… │
│ 3386a93ccaeccd8841c813b09c6f6d… ┆ ["d533d64c76ce33ad87e640fabdcd… │
│ 2a32ffd397739ed866cd345592c51d… ┆ ["f50db1a66557ab24838c0b1c510b… │
│ …                               ┆ …                               │
│ ebfc6e75538f0805cdd838dc7021fc… ┆ ["cfca41d3f37db27ba91afbc96f68… │
│ cfca41d3f37db27ba91afbc96f682b… ┆ ["b43068d7107ad3624b7750a2fd4e… │
│ b43068d7107ad3624b7750a2fd4e09… ┆ ["1

In [32]:
df = pl.scan_parquet("../../data/full_diff.parquet")
df.columns

/var/folders/3b/kmz412j17b5c7z57yc6k1jmm0000gn/T/ipykernel_11789/3357981949.py:2: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df.columns


['commit', 'parent', 'filename', 'insertions', 'deletions']

In [39]:
df.collect().shape

(10052, 5)

In [45]:
result = df.select(
    pl.count("filename").alias("n_filename"),
    pl.col("filename").n_unique().alias("n_unique_filename"),
    pl.col("commit").n_unique().alias("n_unique_commit"),
)
result.collect()

n_filename,n_unique_filename,n_unique_commit
u32,u32,u32
10052,2285,1742


In [70]:
result = df.group_by("filename").agg(
        pl.len().alias("n"),
        pl.col("commit").n_unique().alias("n_commits"),
        pl.col("parent").n_unique().alias("n_parent"),
        pl.col("insertions").sum().alias("total_insertions"),
        pl.col("deletions").sum().alias("total_deletions"),
        # pl.first("last_name"),  # Short for `pl.col("last_name").first()`
).sort("n", descending=True)
# .len(name="n").sort("n", descending=True)
result.collect()

filename,n,n_commits,n_parent,total_insertions,total_deletions
str,u32,u32,u32,u32,u32
"""package.json""",183,183,174,871,596
"""package-lock.json""",126,126,117,429568,476617
"""lib/models/Commit.js""",98,98,98,2278,2014
"""pupil.js""",93,93,92,1523,952
""".github/workflows/docker.yml""",81,81,80,433,432
…,…,…,…,…,…
"""binocular-frontend/src/compone…",1,1,1,0,0
"""binocular-frontend-new/src/plu…",1,1,1,30,0
"""binocular-frontend/src/visuali…",1,1,1,0,0
